In [7]:
# activate the python virtual environment set up for this project -- source linkedin_requester/bin/activate
# open the chrome driver manually - placed in download folder
# have to install selenium -- 
# have to install beautifulsoup -- pip install beautifulsoup4
# have to disable the the 2factor authntication to work smoothly

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import time
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd
from bs4 import BeautifulSoup
import time

In [2]:
# print('Enter your username: ')
# email = input()
# print('Enter your password: ')
# password = input()
# print('Enter the Company Name: ')
# company = input()
# print('Enter the Location: ')
# company = input()
# print('Enter the company name: ')
# company = input()
# print('Enter the Role: ')
# company = input()




In [2]:
# always need to update the chromedriver according to the current chrome version, otherwise will throw-> "SessionNotCreatedException"
# need to start the chrome driver manually figure out how to do it by python script

# driver = webdriver.Chrome(service = Service('/Users/nileshbhoi/downloads/chromedriver'))
# driver = webdriver.Chrome(service = Service('/usr/local/bin/chromedriver'))


# will use firefox for automation as the chrome versoin is updating more and more frequently hard to keep up the chrome webdriver
driver = webdriver.Firefox()

driver.get('https://www.linkedin.com/')
driver


<selenium.webdriver.firefox.webdriver.WebDriver (session="8db6593e-406f-4a96-859e-21b9d50b51fa")>

In [4]:
# click on sign in with google 

EMAIL = ""
PASS = ""

email_elem = driver.find_element(By.ID, 'session_key')
password_elem = driver.find_element(By.ID, 'session_password')
signIn = driver.find_element(By.CSS_SELECTOR, 'button[data-id="sign-in-form__submit-btn"]')

email_elem.send_keys(EMAIL)
password_elem.send_keys(PASS)

signIn.click()

## Start here from main page

In [16]:
from selenium.webdriver.common.keys import Keys

COMPANY = "Intel"
LOCATION = "Seattle, WA"
Role = "Software Engineer"

main_search = driver.find_element(By.CLASS_NAME, "search-global-typeahead__input")
main_search.send_keys(COMPANY)
main_search.send_keys(Keys.ENTER)
time.sleep(2.5) # wait for 10 milli-seconds before procedding


In [17]:
# PEOPLE
driver.find_element(By.XPATH, '//button[text()="People"]').send_keys(Keys.ENTER)
driver.implicitly_wait(10) # wait for 10 milli-seconds before procedding
time.sleep(1)
driver.find_element(By.XPATH, '//button[text()="All filters"]').send_keys(Keys.ENTER)
driver.implicitly_wait(10) # wait for 10 milli-seconds before procedding

In [21]:
# FILTERS
location_filter = driver.find_element(By.XPATH, '//span[text()="Add a location"]')
location_filter.click()
time.sleep(1)

location_enter = driver.find_element(By.XPATH, '//input[@placeholder="Add a location"]')
location_enter.send_keys(LOCATION)
time.sleep(1)

actions = ActionChains(driver)
actions.send_keys(Keys.DOWN).perform()
actions.send_keys(Keys.ENTER).perform()
actions.send_keys(Keys.RETURN).perform()


company_filter = driver.find_element(By.XPATH, '//span[text()="Add a company"]')
company_filter.click()
time.sleep(1)

company_enter = driver.find_element(By.XPATH, '//input[@placeholder="Add a company"]')
company_enter.send_keys(COMPANY)
time.sleep(1)

actions.send_keys(Keys.DOWN).perform()
actions.send_keys(Keys.ENTER).perform()
actions.send_keys(Keys.RETURN).perform()

# attr = f'//span[text()="{COMPANY}"]'
# driver.find_element(By.XPATH, attr).click()
# time.sleep(0.5)

title_element = driver.find_element(By.XPATH, '//label[text()="Title"]')
title_element.find_element(By.TAG_NAME, 'input').send_keys(Role)
# title_element.send_keys(Role)
time.sleep(0.5)

driver.find_element(By.XPATH, '//span[text()="Show results"]').click()

In [22]:
# function to scroll to the bottom of the page
import time
def scroll_to_bottom(driver):
    lenOfPage = driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
    match=False
    while(match==False):
        lastCount = lenOfPage
        time.sleep(0.1)
        lenOfPage = driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
        if lastCount==lenOfPage:
            match=True

In [35]:
# TESTING
# create a list of people

# driver.execute_script("window.scrollTo(0, (document.body.scrollHeight));")
# scroll_to_bottom(driver)

data = BeautifulSoup(driver.page_source, 'html.parser')
pages = data.find(class_="artdeco-pagination__pages artdeco-pagination__pages--number")
# df = pd.DataFrame(columns =['Name', 'Title','Location', 'Link'])


- linkedin has a cap on the number of profiles that you can view, use a ALT account or have to rely on linkedin-premium

In [36]:
df = pd.DataFrame(columns =['Name', 'Title','Location', 'Link'])

# next = driver.find_element(By.XPATH,'//button[@aria-label="Next"]')
i = 0
while True and i<10:
    scroll_to_bottom(driver)
    data = BeautifulSoup(driver.page_source, 'html.parser')
    peoples = data.find_all(class_="entity-result__content entity-result__divider pt3 pb3 t-12 t-black--light")
    list = []
    for people in peoples:
        detail = people.find(class_="entity-result__title-text")
        link = detail.a.get('href')
        if not detail.a.span: # if the name is not unlocked
            continue
        name = detail.a.span.span.get_text()
        title = people.find(class_="entity-result__primary-subtitle t-14 t-black t-normal")
        if  title:
            title = title.get_text()
        else:
            title = "Not Mentioned"
        location = people.find(class_="entity-result__secondary-subtitle t-14 t-normal").get_text()
        list.append([name.strip(), title.strip(), location.strip(), link.strip()])
    temp = pd.DataFrame(list, columns =['Name', 'Title','Location', 'Link'])
    df = pd.concat([df,temp], ignore_index=True)
    # if not pages or i == len(pages.find_all('li'))-1:
    # if not pages :
    #     break
    next = driver.find_element(By.XPATH,'//button[@aria-label="Next"]')
    if not next.is_enabled():
        break
    i += 1
    next.click()
    # driver.implicitly_wait(5) # wait for 10 seconds before procedding
    time.sleep(1.5)

    
df

,Name,Title,Location,Link
0,Jalansh Munshi,Software Engineer @Adobe Firefly,San Francisco Bay Area,https://www.linkedin.com/in/jalansh-munshi?min...
1,Prakruthi Somashekarappa,Software Engineer | Adobe | NCSU,"San Jose, CA",https://www.linkedin.com/in/prakruthi-s?miniPr...
2,Ankur Goyal,Software Engineer at Adobe,"San Jose, CA",https://www.linkedin.com/in/ankurgoyal29?miniP...
3,Jain Zachariah,Software Engineer at Adobe,"San Jose, CA",https://www.linkedin.com/in/jain-zachariah?min...
4,Piyush Mantri,Senior Software Engineer at Adobe,"San Jose, CA",https://www.linkedin.com/in/piyusman?miniProfi...
...,...,...,...,...
91,Zhe Cui,Software Engineer at Adobe,United States,https://www.linkedin.com/in/zhe-cui-831b8b83?m...
92,Kaushik Borra,Sr. Software Development Engineer at Adobe,"San Jose, CA",https://www.linkedin.com/in/saikaushikborra?mi...
93,Ravali Devarapalli,Software Engineer at Adobe,"San Jose, CA",https://www.linkedin.com/in/ravalidevarapalli?...
94,Juhi Singh,Software Development Engineer 2 at Adobe || MS...,"San Jose, CA",https://www.linkedin.com/in/juhisingh39?miniPr...


In [1]:
df.head(10)


NameError: name 'df' is not defined

In [42]:
people = driver.find_elements(By.XPATH,'//span[@class="entity-result__title-line entity-result__title-line--2-lines "]')
people

[<selenium.webdriver.remote.webelement.WebElement (session="1755426a-1f26-4eea-8d6a-1b6b4e784a3a", element="9b11c672-76e8-41dc-a6fc-31795c716b31")>,
 <selenium.webdriver.remote.webelement.WebElement (session="1755426a-1f26-4eea-8d6a-1b6b4e784a3a", element="e9433744-2699-4286-a077-a92822238f99")>,
 <selenium.webdriver.remote.webelement.WebElement (session="1755426a-1f26-4eea-8d6a-1b6b4e784a3a", element="8fb96ad5-1c6b-4195-bb1a-6fcd7f2f827e")>,
 <selenium.webdriver.remote.webelement.WebElement (session="1755426a-1f26-4eea-8d6a-1b6b4e784a3a", element="68dad746-951d-46d5-b3ac-3a9cdab694c1")>,
 <selenium.webdriver.remote.webelement.WebElement (session="1755426a-1f26-4eea-8d6a-1b6b4e784a3a", element="a168a318-58b9-4b48-8726-7c77c49b8999")>,
 <selenium.webdriver.remote.webelement.WebElement (session="1755426a-1f26-4eea-8d6a-1b6b4e784a3a", element="6db1e02c-9712-4de7-abe1-90919a587c81")>,
 <selenium.webdriver.remote.webelement.WebElement (session="1755426a-1f26-4eea-8d6a-1b6b4e784a3a", element

In [41]:
# connect_btn = driver.find_element(By.XPATH, '//span[text()="Connect"]')
# driver.execute_script("arguments[0].click();", connect_btn)
person = driver.find_elements(By.XPATH,'//span[@class="entity-result__title-line entity-result__title-line--2-lines "]')
person[0]



<selenium.webdriver.remote.webelement.WebElement (session="1755426a-1f26-4eea-8d6a-1b6b4e784a3a", element="9b11c672-76e8-41dc-a6fc-31795c716b31")>

In [43]:
while True:
    count = 0
    # scroll_to_bottom(driver)
    while count < 10 and count<(len(driver.find_elements(By.XPATH,'//span[@class="entity-result__title-line entity-result__title-line--2-lines "]'))):
        person = driver.find_elements(By.XPATH,'//span[@class="entity-result__title-line entity-result__title-line--2-lines "]')[count]
        count += 1
        if 'LinkedIn Member' in person.text:
            continue
        person.click()
        time.sleep(1.5)
        driver.back()
    
    
    next = driver.find_element(By.XPATH,'//button[@aria-label="Next"]')
    if not next.is_enabled():
        break
    next.click()

In [25]:
pages = 1
requests = 0
# going over the pages
while True and pages<2:
    # scroll_to_bottom(driver)
    count = 0
    # going over peoples on a page
    while count < 5 and count<(len(driver.find_elements(By.XPATH,'//span[@class="entity-result__title-line entity-result__title-line--2-lines "]'))):
        person = driver.find_elements(By.XPATH,'//span[@class="entity-result__title-line entity-result__title-line--2-lines "]')[count]
        count += 1
        if 'LinkedIn Member' in person.text:
            continue
        person.click()
        # driver.execute_script("arguments[0].click();", person)  
        time.sleep(1.5)
        NAME = driver.find_element(By.XPATH,'//h1[@class="text-heading-xlarge inline t-24 v-align-middle break-words"]').text
        
        TEXT = f"""Hi {NAME},
I'm a MS CS student, with strong MERN stack knowledge. Skilled in problem-solving with Java & Python. Open to learn new technologies.
Could you please refer me for SWE-Intern or connect to a recruiter?
Grateful for any assistance.
-Nilesh"""
        

        more_btn = driver.find_elements(By.XPATH, "//button[@aria-label='More actions']")[1]
        driver.execute_script("arguments[0].click();", more_btn)    

        connect_btn = driver.find_element(By.XPATH, '//span[text()="Connect"]')
        driver.execute_script("arguments[0].click();", connect_btn)

        note_btn = driver.find_element(By.XPATH, '//button[@aria-label="Add a note"]')
        driver.execute_script("arguments[0].click();", note_btn)

        driver.find_element(By.ID, "custom-message").send_keys(TEXT)
        # send_btn = driver.find_element(By.XPATH, '//button[@aria-label="Send now"]')
        close_btn = driver.find_element(By.XPATH, '//button[@class="artdeco-modal__dismiss artdeco-button artdeco-button--circle artdeco-button--muted artdeco-button--2 artdeco-button--tertiary ember-view"]')
        time.sleep(1.5)
        # comment this line and uncomment the line next to it!
        driver.execute_script("arguments[0].click();", close_btn) 
        # # driver.execute_script("arguments[0].click();", send_btn)
        
        requests += 1
        print(f'total requests sent = {requests}-->{NAME}')
        driver.back()
        time.sleep(1)
    # scroll_to_bottom(driver)
    pages += 1
    next = driver.find_element(By.XPATH,'//button[@aria-label="Next"]')
    if not next.is_enabled():
        break
    next.click()
    

total requests sent = 1-->Pranchi Gangil
total requests sent = 2-->Scott Taylor
total requests sent = 3-->Mridul Yadav
total requests sent = 4-->Harshvardhan Chauhan


---

In [3]:
# function to scroll to the bottom of the page
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd
from bs4 import BeautifulSoup
import time
from selenium.webdriver.common.keys import Keys

def scroll_to_bottom(driver):
    lenOfPage = driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
    match=False
    while(match==False):
        lastCount = lenOfPage
        time.sleep(0.1)
        lenOfPage = driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
        if lastCount==lenOfPage:
            match=True

In [ ]:
driver = webdriver.Firefox()

driver.get('https://www.linkedin.com/')

# Signing-in
EMAIL = ""
PASS = ""

email_elem = driver.find_element(By.ID, 'session_key')
password_elem = driver.find_element(By.ID, 'session_password')
signIn = driver.find_element(By.CSS_SELECTOR, 'button[data-id="sign-in-form__submit-btn"]')

email_elem.send_keys(EMAIL)
password_elem.send_keys(PASS)

signIn.click()
# have to split up cause of 2 factor authentication if disabled can be merged into a single cell execution.

In [ ]:

# Wait for the page load
time.sleep(2.5) # wait for 10 milli-seconds before procedding


COMPANY = "Intel"
LOCATION = "Seattle, WA"
Role = "Software Engineer"

main_search = driver.find_element(By.CLASS_NAME, "search-global-typeahead__input")
main_search.send_keys(COMPANY)
main_search.send_keys(Keys.ENTER)
time.sleep(2.5) # wait for 10 milli-seconds before proceeding


# PEOPLE
driver.find_element(By.XPATH, '//button[text()="People"]').send_keys(Keys.ENTER)
driver.implicitly_wait(10) # wait for 10 milli-seconds before procedding
time.sleep(1)
driver.find_element(By.XPATH, '//button[text()="All filters"]').send_keys(Keys.ENTER)
driver.implicitly_wait(10) # wait for 10 milli-seconds before proceeding


# FILTERS
location_filter = driver.find_element(By.XPATH, '//span[text()="Add a location"]')
location_filter.click()
time.sleep(1)

location_enter = driver.find_element(By.XPATH, '//input[@placeholder="Add a location"]')
location_enter.send_keys(LOCATION)
time.sleep(1)

actions = ActionChains(driver)
actions.send_keys(Keys.DOWN).perform()
actions.send_keys(Keys.ENTER).perform()
actions.send_keys(Keys.RETURN).perform()


company_filter = driver.find_element(By.XPATH, '//span[text()="Add a company"]')
company_filter.click()
time.sleep(1)

company_enter = driver.find_element(By.XPATH, '//input[@placeholder="Add a company"]')
company_enter.send_keys(COMPANY)
time.sleep(1)

actions.send_keys(Keys.DOWN).perform()
actions.send_keys(Keys.ENTER).perform()
actions.send_keys(Keys.RETURN).perform()

title_element = driver.find_element(By.XPATH, '//label[text()="Title"]')
title_element.find_element(By.TAG_NAME, 'input').send_keys(Role)
time.sleep(0.5)

driver.find_element(By.XPATH, '//span[text()="Show results"]').click()